In [5]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length=2048
dtype=None
load_in_4bit=True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset
dataset = load_dataset(
    "csv",
    data_files = "https://huggingface.co/datasets/unsloth/datasets/raw/main/titanic.csv",
    split = "train",
)
print(dataset.column_names)
print(dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
{'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': None, 'Embarked': 'S'}


In [6]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt=\
    "[[The passenger embarked from {Embarked}.]]"\
    "[[\nThey are{Sex}.]]"\
    "[[\nThey have {Parch} parents and children.]]"\
    "[[\nThey have {SibSp} siblings and spouses.]]"\
    "[[\nThey are {Age} years old.]]"\
    "[[\nThey paid ${Fare} for their ticket.]]",
    conversation_extension=5,
    output_column_name="Survived",


    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Merging columns:   0%|          | 0/891 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/891 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/891 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/891 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/891 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/891 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/891 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/891 [00:00<?, ? examples/s]

In [11]:
dataset['conversations'][0]

[{'content': 'The passenger embarked from S.\nThey aremale.\nThey have 1 siblings and spouses.\nThey are 22.0 years old.\nThey paid $7.25 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from S.\nThey aremale.\nThey have 1 parents and children.\nThey have 1 siblings and spouses.\nThey are 52.0 years old.\nThey paid $79.65 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from S.\nThey arefemale.\nThey have 2 parents and children.\nThey have 4 siblings and spouses.\nThey are 9.0 years old.\nThey paid $31.275 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from C.\nThey aremale.\nThey paid $7.8958 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from S.\nThey arefemale.\nThey are 24.0 years old.\nThey paid $13.0 for their ti

In [9]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Standardizing format:   0%|          | 0/891 [00:00<?, ? examples/s]

In [12]:
dataset['conversations'][0]

[{'content': 'The passenger embarked from S.\nThey aremale.\nThey have 1 siblings and spouses.\nThey are 22.0 years old.\nThey paid $7.25 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from S.\nThey aremale.\nThey have 1 parents and children.\nThey have 1 siblings and spouses.\nThey are 52.0 years old.\nThey paid $79.65 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from S.\nThey arefemale.\nThey have 2 parents and children.\nThey have 4 siblings and spouses.\nThey are 9.0 years old.\nThey paid $31.275 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from C.\nThey aremale.\nThey paid $7.8958 for their ticket.',
  'role': 'user'},
 {'content': '0', 'role': 'assistant'},
 {'content': 'The passenger embarked from S.\nThey arefemale.\nThey are 24.0 years old.\nThey paid $13.0 for their ti

In [13]:
chat_template = """Below describes some details about some passengers who went on the Titanic.
Predict whether they survived or perished based on their characteristics.
Output 1 if they survived, and 0 if they died.
>>> Passenger Details:
{INPUT}
>>> Did they survive?
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

NameError: name 'tokenizer' is not defined

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [22]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/891 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 891 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.499800
2,1.463700
3,1.438700
4,1.297300
5,1.128600
6,0.957400
7,0.805300
8,0.676100
9,0.582000
10,0.511700


In [24]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": 'The passenger embarked from S.\n'\
                                'They are male.\n'\
                                'They have 1 siblings and spouses.\n'\
                                'Their passenger class is 3.\n'\
                                'Their age is 22.0.\n'\
                                'They paid $7.25 for the trip.'},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0<|end_of_text|>


In [25]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": 'Their passenger class is 1.\n'\
                                'Their age is 22.0.\n'\
                                'They paid $107.25 for the trip.'},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

1<|end_of_text|>


In [26]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [29]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

messages = [                    # Change below!
    {"role": "user", "content": 'Their passenger class is 3.\n'\
                                'Their age is 22.0.\n'\
                                'They paid $107.25 for the trip.'},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 